In [ ]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
system_message = "You are an assistant that views pull request raised by software Python developer code. "
system_message += "Respond only with python code; use comments sparingly and do not provide any explanation other than occasional comments. "



In [ ]:
def user_prompt_for(python):
    user_prompt = "Fomrat this Python code using PEP8 style put doctstring by understanding the module or function whichever is applicable. "
    user_prompt += "Respond only with python code; with proper clean and elagant. "
    user_prompt += python
    return user_prompt

In [ ]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
def write_output(python_code):
    python_code = python_code.replace('```python\n','').replace('```','')
    with open("pep8_formatted.py", "w") as f:
        f.write(python_code)

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')
    write_output(reply)
    

In [ ]:
def call_model(python, model):  
    
    reply = ""
    if model == 'GPT':
        stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    elif model == 'Claude':
        stream = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    elif model == 'CodeQwen':
        tokenizer = AutoTokenizer.from_pretrained(code_qwen)
        messages = messages_for(python)
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        client = InferenceClient(CODE_QWEN_URL, token=hf_token)
        stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
        yield reply.replace('```python\n','').replace('```','')
    write_output(reply)

In [ ]:
sample = """
import random

def start_game():

    mat =[]
    for i in range(4):
        mat.append([0] * 4)


    print("Commands are as follows : ")
    print("'W' or 'w' : Move Up")
    print("'S' or 's' : Move Down")
    print("'A' or 'a' : Move Left")
    print("'D' or 'd' : Move Right")

    add_new_2(mat)
    return mat

def findEmpty(mat):
    for i in range(4):
        for j in range(4):
            if mat[i][j] == 0:
                return i, j 
    return None, None
"""

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=sample)
        cpp = gr.Textbox(label="PEP8 formatted code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
        convert = gr.Button("Fomrat code")

    convert.click(call_model, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)